In [34]:
import torch as t
import numpy as np

从接口的角度来讲，对Tensor的操作可分为两类：

- torch.function，如torch.save等。
- tensor.function，如tensor.view等。

为方便使用，对Tensor的大部分操作同时支持这两类接口，在本书中不做具体区分，如`torch.sum(a, b)与a.sum(b)`功能等价。

而从存储的角度来讲，对Tensor的操作又可分为两类：

- 不会修改自身的数据，如 `a.add(b)`， 加法的结果会返回一个新的Tensor。
- 会修改自身的数据，如 `a.add_(b)`， 加法的结果仍存储在a中，a被修改了。

函数名以_结尾的都是inplace方式，即会修改调用者自己的数据，在实际应用中需加以区分。

## Tensor基本操作

list转tensor

In [23]:
data = [[1, 2],[3, 4]]
a = t.tensor(data) #生成以一个和之前形状一样的tensor
c = t.Tensor() # 直接创建空tensor
d = t.tensor(())
print(a)
print(c)
print(d)

#list转list
b = a.tolist()
print(b) #将tensor转化回list

tensor([[1, 2],
        [3, 4]])
tensor([])
tensor([])
[[1, 2], [3, 4]]


x_like初始化

In [12]:
x_data = t.tensor([[1,2],[3,4]])
x_ones = t.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = t.rand_like(x_data, dtype=t.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.6879, 0.8007],
        [0.7155, 0.1374]]) 



产生特定形状的tensor

In [ ]:
shape = (2,3)
one_data = t.ones(shape)
rand_data = t.rand(shape)
zero_data = t.zeros(shape)
randint_data = t.randint(3, 10, shape)
print(f"Random Tensor: \n {randint_data} \n")
print(f"one tensor: {one_data}")
print(f"zero tensor: {zero_data}")
print(f"randint tensor: {randint_data}")

Random Tensor: 
 tensor([[5, 5, 6],
        [9, 3, 4]]) 

one tensor: tensor([[1., 1., 1.],
        [1., 1., 1.]])
zero tensor: tensor([[0., 0., 0.],
        [0., 0., 0.]])
randint tensor: tensor([[5, 5, 6],
        [9, 3, 4]])


In [27]:
tensor3 = t.tensor((), dtype=t.int32) #通过指定对应的类型
tensor3.new_ones((2, 3)) #对空tensor进行重新设置

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

产生随机排列的tensor

In [40]:
# 创建一个长度为8的随机排列的Tensor
t.randperm(8) 

tensor([1, 4, 2, 6, 3, 7, 5, 0])

产生对角线为1，其余为0的tensor

In [46]:
# 创建一个对角线为1，其余为0的Tensor,不要求行列数一致
t.eye(2, 3, dtype=t.int)

tensor([[1, 0, 0],
        [0, 1, 0]], dtype=torch.int32)

In [47]:
# 创建一个起始值为1，上限为6，步长为2的Tensor
t.arange(1, 6, 2)

tensor([1, 3, 5])

In [48]:
# 创建一个均匀间距的Tensor，将1到10的数分为3份
t.linspace(1, 10, 3)

tensor([ 1.0000,  5.5000, 10.0000])

tensor元素个数计算

In [9]:
x_data = t.randn(2,3)
print(x_data.numel())
print(x_data.nelement())

6
6


tensor的形状

In [10]:
print(x_data.size())
print(x_data.shape)

torch.Size([2, 3])
torch.Size([2, 3])


## Tensor属性

In [25]:
tensor = t.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")
print(f"tensor 's type:{tensor.type()}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu
tensor 's type:torch.FloatTensor


## Tensor类型

表3-2: Tensor数据类型

| Data type       | dtype                        | CPU tensor         | GPU tensor            |
|-----------------|------------------------------|--------------------|----------------------|
| 32-bit浮点型    | `torch.float32` or `torch.float` | `torch.FloatTensor` | `torch.cuda.FloatTensor` |
| 64-bit浮点型    | `torch.float64` or `torch.double` | `torch.DoubleTensor` | `torch.cuda.DoubleTensor` |
| 16-bit半精度浮点型 | `torch.float16` or `torch.half` | `torch.HalfTensor` | `torch.cuda.HalfTensor` |
| 8-bit无符号整型 | `torch.uint8`                | `torch.ByteTensor` | `torch.cuda.ByteTensor` |
| 8-bit有符号整型 | `torch.int8`                 | `torch.CharTensor` | `torch.cuda.CharTensor` |
| 16-bit有符号整型 | `torch.int16` or `torch.short` | `torch.ShortTensor` | `torch.cuda.ShortTensor` |
| 32-bit有符号整型 | `torch.int32` or `torch.int` | `torch.IntTensor` | `torch.cuda.IntTensor` |
| 64-bit有符号整型 | `torch.int64` or `torch.long` | `torch.LongTensor` | `torch.cuda.LongTensor` |

不同Tensor类型之间的转换方法

1. **数据类型转换**：各种数据类型之间可以互相转换，`type(new_type)` 是通用的做法，同时还有 `float`、`long`、`half` 等快捷方法。

2. **CPU与GPU Tensor转换**：CPU Tensor与GPU Tensor之间的互相转换通过 `tensor.cuda` 和 `tensor.cpu` 方法实现，此外还可以使用 `tensor.to(device)`。

3. **创建同种类型的张量**：
   - 使用 `torch.*_like` 和 `tensor.new_*`。例如，`torch.ones_like()` 可以生成和 `tensorA` 拥有同样属性（类型，形状，CPU/GPU）的新Tensor。
   - `tensor.new_*(new_shape)` 可以新建一个不同形状的Tensor，例如 `tensor.new_zeros()`。

## 判断cuda是否可用

In [14]:
if t.cuda.is_available():
    x_data = x_data.to("cuda")
    device = "cuda"

## 切片和索引操作

In [15]:
tensor = t.ones(4, 4)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

First row:  tensor([1., 1., 1., 1.])
First column:  tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [41]:
a = t.randn(3,4)
print(a)

tensor([[-0.0957,  0.1117, -0.1136, -1.3344],
        [ 0.6497, -1.6803,  0.0482, -0.4707],
        [-1.1186, -0.1281, -1.1201,  0.4486]])


In [42]:
print("查看第1行结果：", a[0])
print("查看第2列结果：", a[:,1])
print("查看第2行最后两个元素：", a[1, -2:])

查看第1行结果： tensor([-0.0957,  0.1117, -0.1136, -1.3344])
查看第2列结果： tensor([ 0.1117, -1.6803, -0.1281])
查看第2行最后两个元素： tensor([ 0.0482, -0.4707])


In [43]:
# 返回一个ByteTensor
print(a > 0) # bool型
print((a > 0).int()) # 整型

tensor([[False,  True, False, False],
        [ True, False,  True, False],
        [False, False, False,  True]])
tensor([[0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 0, 0, 1]], dtype=torch.int32)


In [45]:
# 返回Tensor中满足条件的结果，下面两种写法等价
# 选择返回的结果与原Tensor不共享内存空间
print(a[a > 0])
print(a.masked_select(a>0)) 
print(a)

# 用torch.where保留原始的索引位置，不满足条件的位置置0
print(t.where(a > 0, a, t.zeros_like(a)))

tensor([0.1117, 0.6497, 0.0482, 0.4486])
tensor([0.1117, 0.6497, 0.0482, 0.4486])
tensor([[-0.0957,  0.1117, -0.1136, -1.3344],
        [ 0.6497, -1.6803,  0.0482, -0.4707],
        [-1.1186, -0.1281, -1.1201,  0.4486]])
tensor([[0.0000, 0.1117, 0.0000, 0.0000],
        [0.6497, 0.0000, 0.0482, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4486]])


view的操作，将原来的tensor的形状转化为4*4的形状

In [ ]:

a = t.arange(0, 16).view(4, 4)
print(a)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])


取对角线上的元素

In [ ]:
index = t.tensor([[0,1,2,3]])
a.gather(0, index) #dim = 0表示行

tensor([[ 0,  5, 10, 15]])

选取反对角线上的元素

In [53]:
# 选取反对角线上的元素
index = t.tensor([[3,2,1,0]]).t()
a.gather(1, index) #dim = 1 表示对列进行提取

tensor([[ 3],
        [ 6],
        [ 9],
        [12]])

选取两条对角线上的元素

In [52]:
# 选取两个对角线上的元素
index = t.tensor([[0,1,2,3],[3,2,1,0]]).t()
b = a.gather(1, index)
b

tensor([[ 0,  3],
        [ 5,  6],
        [10,  9],
        [15, 12]])

## 高级索引

In [54]:
x = t.arange(0,16).view(2,2,4)
x

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]]])

In [59]:
# 从一个三维的tensor中获取对应位置的元素可以表示为：
print(x[0,1,2]) #表示第0行第1列第2个元素

print(x[[1,0],[1,1],[2,0]]) #表示第1行第1列第2个元素和第0行第1列第0个元素

print(x[[1, 0], [0], [1]]) #表示x[1,0,1]和x[0,0,1], 如果对应位置只有一个数字表示都采用这个数字

tensor(6)
tensor([14,  4])
tensor([9, 1])


## 逐元素操作

In [60]:
a = t.arange(0, 6).float().view(2, 3)
t.cos(a)

tensor([[ 1.0000,  0.5403, -0.4161],
        [-0.9900, -0.6536,  0.2837]])

In [ ]:
# 表示取模操作，下面两种写法等价
print(a % 3)
print(t.fmod(a, 3))

tensor([[0., 1., 2.],
        [0., 1., 2.]])
tensor([[0., 1., 2.],
        [0., 1., 2.]])


In [63]:
# 对tensor元素进行上下限截断保持在给定的范围中
print(a)
print(t.clamp(a, min=1, max=3))

tensor([[0., 1., 2.],
        [3., 4., 5.]])
tensor([[1., 1., 2.],
        [3., 3., 3.]])


## 归并操作

归并操作会使用Tensor中的部分元素进行计算，其输出形状可能小于输入形状。同时我们可以沿着某一维度进行指定的归并操作。如加法`sum`，既可以计算整个Tensor的和，也可以计算Tensor中每一行或每一列的和。常用的归并操作如表3-5所示。

假设输入的形状是(m, n, k)

- 如果指定dim=0，输出的形状就是(1, n, k)或者(n, k)
- 如果指定dim=1，输出的形状就是(m, 1, k)或者(m, k)
- 如果指定dim=2，输出的形状就是(m, n, 1)或者(m, n)

其中size中是否有`"1"`，取决于参数`keepdim`，若指定`keepdim=True`则会保留维度1

In [64]:
b = t.ones(2, 3)
print(b)

tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [ ]:
# 注意对比是否保留维度1的区别
print(b.sum(dim=0, keepdim=True ),b.sum(dim=0, keepdim=True ).shape) #dim=0表示对列进行求和，相当于是保留下了列
print(b.sum(dim=0, keepdim=False), b.sum(dim=0, keepdim=False).shape)

tensor([[2., 2., 2.]]) torch.Size([1, 3])
tensor([2., 2., 2.]) torch.Size([3])


cumsum这个操作相当于是按照行或者列 累加一直堆下去

In [66]:
a = t.arange(2, 8).view(2, 3)
print(a)
print(a.cumsum(dim=1)) # 沿着行累加

tensor([[2, 3, 4],
        [5, 6, 7]])
tensor([[ 2,  5,  9],
        [ 5, 11, 18]])


In [67]:
a = t.linspace(0, 15, 6).view(2, 3)
b = t.linspace(15, 0, 6).view(2, 3)
print(a > b)
print("a中大于b的元素: ", a[a > b]) # 返回a中大于b的元素
print("a中最大的元素: ", t.max(a)) # 返回a中最大的元素

tensor([[False, False, False],
        [ True,  True,  True]])
a中大于b的元素:  tensor([ 9., 12., 15.])
a中最大的元素:  tensor(15.)


In [ ]:
print(b)
print(t.max(b, dim = 0)) # 返回b中每一列最大的元素

tensor([[15., 12.,  9.],
        [ 6.,  3.,  0.]])
torch.return_types.max(
values=tensor([15., 12.,  9.]),
indices=tensor([0, 0, 0]))


In [75]:
t.max(a, b) # 两个Tensor对应位置上较大的元素
# 注意这个要求必须两个tensor的大小是一样的才有可比性

tensor([[15., 12.,  9.],
        [ 9., 12., 15.]])

## 拼接操作

In [16]:
x_ten= t.rand((2,3))
t1 = t.cat([x_ten, x_ten, x_ten], dim=1) #dim = 1表示增加列
print(t1) 
t2 = t.cat([x_ten, x_ten, x_ten], dim=0) #dim = 0表示增加行
print(t2) 

tensor([[0.1189, 0.4205, 0.8536, 0.1189, 0.4205, 0.8536, 0.1189, 0.4205, 0.8536],
        [0.3248, 0.4944, 0.5703, 0.3248, 0.4944, 0.5703, 0.3248, 0.4944, 0.5703]])
tensor([[0.1189, 0.4205, 0.8536],
        [0.3248, 0.4944, 0.5703],
        [0.1189, 0.4205, 0.8536],
        [0.3248, 0.4944, 0.5703],
        [0.1189, 0.4205, 0.8536],
        [0.3248, 0.4944, 0.5703]])


## 算术运算
 

In [17]:
tensor = t.rand(2, 3)
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T) #表示矩阵乘法
print(y1)
print(y2)

z1 = tensor * tensor
z2 = tensor.mul(tensor) #表示矩阵中按照element-wise进行乘法运算
print(z1)
print(z2)

tensor([[0.1851, 0.2463],
        [0.2463, 0.3928]])
tensor([[0.1851, 0.2463],
        [0.2463, 0.3928]])
tensor([[0.0808, 0.0796, 0.0247],
        [0.1023, 0.0912, 0.1993]])
tensor([[0.0808, 0.0796, 0.0247],
        [0.1023, 0.0912, 0.1993]])


## 运算元素查询

In [16]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

3.6891863346099854 <class 'float'>


## 元素替换 in-place

In [24]:
tensor = t.rand((2,3))
print(tensor, "\n")
tensor.add_(5)
print(tensor)
tensor_y = t.rand((2,3))
tensor_y.copy_(tensor) #这个操作必须是初始化之后才能使用
print(tensor_y)

tensor([[0.9854, 0.7586, 0.3820],
        [0.1077, 0.9946, 0.5401]]) 

tensor([[5.9854, 5.7586, 5.3820],
        [5.1077, 5.9946, 5.5401]])
tensor([[5.9854, 5.7586, 5.3820],
        [5.1077, 5.9946, 5.5401]])


In [21]:
print(tensor_y)
tensor_y[:,0].fill_(4) #将第一列的元素全部替换成4，inplace操作
print(tensor_y)

tensor([[4.0000, 5.1970, 5.5264],
        [4.0000, 5.7325, 5.1879]])
tensor([[4.0000, 5.1970, 5.5264],
        [4.0000, 5.7325, 5.1879]])


## Tensor与Numpy

In [37]:
th = t.ones(5)
print(f"t: {th}")
n = th.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [ ]:
th.add_(1)
print(f"t: {th}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]


Numpy转Tensor

In [28]:
n = np.ones(5)
t = t.from_numpy(n)

In [25]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]


## 改变tensor的维度

查看tensor对应的性质属性

In [77]:
tensor = t.arange(24).reshape(2, 3, 4)
# tensor.shape 和tensor.size() 等价
print(f"a.shape={tensor.shape}. a.size()={tensor.size()}")
print(f'这是个{tensor.dim()}维Tensor, 总共{tensor.numel()}个元素')

a.shape=torch.Size([2, 3, 4]). a.size()=torch.Size([2, 3, 4])
这是个3维Tensor, 总共24个元素


矩阵的转置会导致存储空间不连续，因此需调用它的.contiguous方法将其转为连续

In [80]:
a = t.linspace(0, 15, 6).view(2, 3)
print(a)
b = a.t()
print(b)
b.is_contiguous()


tensor([[ 0.,  3.,  6.],
        [ 9., 12., 15.]])
tensor([[ 0.,  9.],
        [ 3., 12.],
        [ 6., 15.]])


False

In [82]:
b = b.contiguous()
b.is_contiguous()

True

所有改变Tensor形状的操作都可以通过tensor.reshape实现。`tensor.reshape(new_shape)`会把不连续的`Tensor`变成连续的再进行形状变化，这一操作等价于`tensor.contiguous().view(new_shape)`。关于view和reshape的选用可参考下面的建议:

- 对于reshape而言:其接口更便捷，`会自动把不连续的Tensor变为连续的`，能避免很多报错。同时它的函数名和NumPy一样，便于使用。
- 对于view而言:其函数名更短，而且Tensor经过view操作之后仍然共享存储空间。当我们并不希望改变形状之后的Tensor和原Tensor共享存储时可以使用view方法。
  
reshape和view二者之间有一些区别：如果对Tensor使用了transpose、permute等操作，会造成Tensor的内存变得不连续，而view方法只能改变连续的张量，所以需要先调用`contiguous()`方法，但reshape方法则不受此限制。具体来说，view方法返回的Tensor和原Tensor共享Storage（注意：不是共享内存地址），而reshape方法可能返回原Tensor的copy或者共享Storage的view。如果当前满足连续性条件，则结果与view相同；否则返回的就是copy（此时等价于`tensor.contiguous().view()`）。

In [85]:
a = t.arange(1, 13)
b = a.view(2, 6)
c = a.reshape(2, 6) # 此时view和reshape等价，因为Tensor是contiguous
print(a.shape, b.shape, c.shape)

torch.Size([12]) torch.Size([2, 6]) torch.Size([2, 6])


In [ ]:
b = b.t() # b 不再连续
print(b.reshape(-1, 4)) # reshape 可以

# 下面会报错，view无法在改变数据存储的情况下进行形状变化
# b.view(-1, 4)
print(b.contiguous().view(-1,4))
# 注意这里的改变形状都不是本身改变的，而是产生了一个新的tensor

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])


reshape操作的难点在于如何快速灵活地指定形状，常用的快捷变形方法有：

- `tensor.view(dim1,-1,dimN)`: 在调整Tensor形状的时候，我们不需要指定每一维的形状，可以把其中一个维度指定为-1，PyTorch会自动计算对应的形状
- `tensor.view_as(other)`: 将Tensor的形状变为和other 一样，等价于tensor.view(other.shape)
- `tensor.squeeze()`: 将Tensor形状中为1的维度去掉，比如(1,3,1,4)变为(3,4)
- `tensor.flatten(start_dim=0, end_dim=-1)`: 将Tensor形状中某些连续维度合并为一个维度. 比如形状(2,3,4,5) ->(2,12,5)。
- `tensor[None]` 和 `tensor.unsqueeze(dim)`: 为Tensor新建一个维度，并把形状设为1, 比如形状(2,3) -> (2,1,3)

In [95]:
H, W = 4, 5
img_3xHxW = t.randn(3, H, W)

# 目标数据排列和输入一样，直接使用reshape
img_3xHW = img_3xHxW.reshape(3, -1)
print(img_3xHW.shape)

# 目标数据排列和输入不一样，先通过transpose变成(3,W,H), 再变成(3,WH)
img_3xWH = img_3xHxW.transpose(1, 2)
print(img_3xHW.shape, img_3xWH.shape)

img_a = img_3xWH.unsqueeze(dim = 2)
print(img_a.shape)

img_b = img_a.squeeze(dim = 2).flatten(start_dim = 1, end_dim = -1)
print(img_b.shape)

# 再变形为3xWxH的形式
img_3xWxH = img_3xWH.reshape(3, W, H)

torch.Size([3, 20])
torch.Size([3, 20]) torch.Size([3, 5, 4])
torch.Size([3, 5, 1, 4])
torch.Size([3, 20])
